In [8]:
# LTCI Cost of Waiting 

In [13]:
#from financial_planner.entities.term_structure import term_structure
from financial_planner.entities.household import Person


ModuleNotFoundError: No module named 'financial_planner.entities'

In [2]:
class  LTCI_cost_of_waiting():
    '''Calculate the Potential Cost of Waiting to Purchase a Long-Term-Care Insurance Policy
    '''
    def __init__(current_age, purchase_age, benifit_age, current_premium, future_premium):
        '''
        current_age, is the age current age of the customer
        purchase_age, is the age the customer plan to enter a insurance policy
        benifit_age, is the age the customer will receive the benifit from the insurance
        current_premium, is the annual premium if the customer enters the insurance right now
        future_premium, is the annual premium if the customer enters the insurance at the purchase_age
        '''
        self.current_age = current_age
        self.purchase_age = purchase_age
        self.benifit_age = benifit_age
        self.current_premium = current_premium
        self.future_premium = future_premium
        
    def discount_factor(nyears):
        nyears = self.benifit_age-self.current_age
        term_stracture = 
        discount_f = 
        return discount_f
        
    def furture_cashflow():
        fu_cashflow = np.concatenate(np.zeros(self.purchase_age-self.current_age),
                                     np.repeat(self.future_premium,(self.benifit_age-self.purchase_age)))
        return fu_cashflow
        
    def current_cashflow():
        cu_cashflow = np.concatenate(np.repeat(self.current_premium,(self.benifit_age-self.current_age)))

    
    def cost_future_purchase():
        dis_factor = discount_factor()
        cost_future_purchase = sum(dis_factor*furture_cashflow)
   
    def cost_current_purchase():

        cost_current_purchase = sum(dis_factor*current_cashflow)
    
    def cost():
        waiting_cost = cost_future_purchase()-cost_current_purchase()


In [4]:
(80-55)*1480-(80-56)*1598

-1352